<a href="https://colab.research.google.com/github/sgriesmer/njit-cad/blob/main/Training_DNABERT_Classifier_With_TFBS_Datasets_streamlined_sjg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
!pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 103.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.2 MB/s eta 0:00:00


Access TFBS dataset in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Run model on next 10 datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_metric
from transformers import TrainingArguments, Trainer
import numpy as np
import sys
import os

# initialize parameters

for fname in ["HaibGm12878Zbtb33Pcr1xUniPk.csv",
  "HaibGm12878Zeb1sc25388V0416102UniPk.csv",
  "HaibGm12891Pax5c20V0416101UniPk.csv",
  "HaibGm12891Pol24h8Pcr1xUniPk.csv",
  "HaibGm12891Pol2Pcr1xUniPk.csv",
  "HaibGm12891Pou2f2Pcr1xUniPk.csv",
  "HaibGm12891Pu1Pcr1xUniPk.csv",
  "HaibGm12891Taf1Pcr1xUniPk.csv",
  "HaibGm12891Yy1sc281V0416101UniPk.csv",
  "HaibGm12892Pax5c20V0416101UniPk.csv"]:
#  "HaibGm12892Pol24h8V0416102UniPk.csv",
#  "HaibGm12892Pol2V0416102UniPk.csv",
#  "HaibGm12892Taf1V0416102UniPk.csv",
#  "HaibGm12892Yy1V0416101UniPk.csv",
#  "HaibH1hescAtf2sc81188V0422111UniPk.csv",
#  "HaibH1hescAtf3V0416102UniPk.csv",
#  "HaibH1hescBcl11aPcr1xUniPk.csv",
#  "HaibH1hescCtcfsc5916V0416102UniPk.csv",
#  "HaibH1hescEgr1V0416102UniPk.csv",
#  "HaibH1hescFosl1sc183V0416102UniPk.csv",
#  "HaibH1hescGabpPcr1xUniPk.csv",
#  "HaibH1hescHdac2sc6296V0416102UniPk.csv",
#  "HaibH1hescJundV0416102UniPk.csv",
#  "HaibH1hescNanogsc33759V0416102UniPk.csv",
#  "HaibH1hescNrsfV0416102UniPk.csv",
#  "HaibH1hescP300V0416102UniPk.csv",
#  "HaibH1hescPol24h8V0416102UniPk.csv",
#  "HaibH1hescPol2V0416102UniPk.csv",
#  "HaibH1hescPou5f1sc9081V0416102UniPk.csv",
#  "HaibH1hescRad21V0416102UniPk.csv",
#  "HaibH1hescRxraV0416102UniPk.csv",
#  "HaibH1hescSin3ak20Pcr1xUniPk.csv",
#  "HaibH1hescSix5Pcr1xUniPk.csv",
#  "HaibH1hescSp1Pcr1xUniPk.csv",
#  "HaibH1hescSp2V0422111UniPk.csv",
#  "HaibH1hescSp4v20V0422111UniPk.csv",
#  "HaibH1hescSrfPcr1xUniPk.csv",
#  "HaibH1hescTaf1V0416102UniPk.csv",
#  "HaibH1hescTaf7sc101167V0416102UniPk.csv",
#  "HaibH1hescTcf12Pcr1xUniPk.csv",
#  "HaibH1hescTead4sc101184V0422111UniPk.csv",
#  "HaibH1hescUsf1Pcr1xUniPk.csv",
#  "HaibH1hescYy1sc281V0416102UniPk.csv",
#  "HaibHct116Pol24h8V0416101UniPk.csv",
#  "HaibHct116Yy1sc281V0416101UniPk.csv",
#  "HaibHct116Zbtb33V0416101UniPk.csv",
#  "HaibHelas3GabpPcr1xUniPk.csv",
#  "HaibHelas3NrsfPcr1xUniPk.csv",
#  "HaibHelas3Pol2Pcr1xUniPk.csv",
#  "HaibHelas3Taf1Pcr1xUniPk.csv",
#  "HaibHepg2Atf3V0416101UniPk.csv",
#  "HaibHepg2Bhlhe40V0416101UniPk.csv",
#  "HaibHepg2Cebpbsc150V0416101UniPk.csv",
#  "HaibHepg2Cebpdsc636V0416101UniPk.csv",
#  "HaibHepg2Ctcfsc5916V0416101UniPk.csv",
#  "HaibHepg2Elf1sc631V0416101UniPk.csv",
#  "HaibHepg2Fosl2V0416101UniPk.csv",
#  "HaibHepg2Foxa1sc101058V0416101UniPk.csv",
#  "HaibHepg2Foxa1sc6553V0416101UniPk.csv",
#  "HaibHepg2Foxa2sc6554V0416101UniPk.csv",
  # "HaibHepg2GabpPcr2xUniPk.csv",
  # "HaibHepg2Hdac2sc6296V0416101UniPk.csv",
  # "HaibHepg2Hnf4asc8987V0416101UniPk.csv",
  # "HaibHepg2Hnf4gsc6558V0416101UniPk.csv",
  # "HaibHepg2JundPcr1xUniPk.csv",
  # "HaibHepg2Mbd4sc271530V0422111UniPk.csv",
  # "HaibHepg2Mybl2sc81192V0422111UniPk.csv",
  # "HaibHepg2Nficsc81335V0422111UniPk.csv",
  # "HaibHepg2NrsfPcr2xUniPk.csv",
  # "HaibHepg2NrsfV0416101UniPk.csv",
  # "HaibHepg2P300V0416101UniPk.csv",
  # "HaibHepg2Pol24h8V0416102UniPk.csv",
  # "HaibHepg2Pol2Pcr2xUniPk.csv",
  # "HaibHepg2Rad21V0416101UniPk.csv",
  # "HaibHepg2RxraPcr1xUniPk.csv",
  # "HaibHepg2Sin3ak20Pcr1xUniPk.csv",
  # "HaibHepg2Sp1Pcr1xUniPk.csv",
  # "HaibHepg2Sp2V0422111UniPk.csv",
  # "HaibHepg2SrfV0416101UniPk.csv",
  # "HaibHepg2Taf1Pcr2xUniPk.csv",
  # "HaibHepg2Tcf12Pcr1xUniPk.csv",
  # "HaibHepg2Tead4sc101184V0422111UniPk.csv",
  # "HaibHepg2Usf1Pcr1xUniPk.csv",
  # "HaibHepg2Yy1sc281V0416101UniPk.csv",
  # "HaibHepg2Zbtb33Pcr1xUniPk.csv",
  # "HaibHepg2Zbtb7aV0416101UniPk.csv",
  # "HaibHuvecPol24h8V0416101UniPk.csv",
  # "HaibHuvecPol2Pcr1xUniPk.csv",
  # "HaibK562Atf3V0416101UniPk.csv",
  # "HaibK562Bcl3Pcr1xUniPk.csv",
  # "HaibK562Bclaf101388Pcr1xUniPk.csv",
  # "HaibK562Cbx3sc101004V0422111UniPk.csv",
  # "HaibK562Cebpbsc150V0422111UniPk.csv",
  # "HaibK562CtcfcPcr1xUniPk.csv",
  # "HaibK562Ctcflsc98982V0416101UniPk.csv",
  # "HaibK562E2f6V0416102UniPk.csv",
  # "HaibK562Egr1V0416101UniPk.csv",
  # "HaibK562Elf1sc631V0416102UniPk.csv",
  # "HaibK562Ets1V0416101UniPk.csv",
  # "HaibK562Fosl1sc183V0416101UniPk.csv",
  # "HaibK562GabpV0416101UniPk.csv",
  # "HaibK562Gata2sc267Pcr1xUniPk.csv",
  # "HaibK562Hdac2sc6296V0416102UniPk.csv",
  # "HaibK562MaxV0416102UniPk.csv",
  # "HaibK562Mef2aV0416101UniPk.csv",
  # "HaibK562Nr2f2sc271940V0422111UniPk.csv",
  # "HaibK562NrsfV0416102UniPk.csv",
  # "HaibK562Pmlsc71910V0422111UniPk.csv",
  # "HaibK562Pol24h8V0416101UniPk.csv",
  # "HaibK562Pol2V0416101UniPk.csv",
  # "HaibK562Pu1Pcr1xUniPk.csv",
  # "HaibK562Rad21V0416102UniPk.csv",
  # "HaibK562Sin3ak20V0416101UniPk.csv",
  # "HaibK562Six5Pcr1xUniPk.csv",
  # "HaibK562Sp1Pcr1xUniPk.csv",
  # "HaibK562Sp2sc643V0416102UniPk.csv",
  # "HaibK562SrfV0416101UniPk.csv",
  # "HaibK562Stat5asc74442V0422111UniPk.csv",
  # "HaibK562Taf1V0416101UniPk.csv",
  # "HaibK562Taf7sc101167V0416101UniPk.csv",
  # "HaibK562Tead4sc101184V0422111UniPk.csv",
  # "HaibK562Thap1sc98174V0416101UniPk.csv",
  # "HaibK562Trim28sc81411V0422111UniPk.csv",
  # "HaibK562Usf1V0416101UniPk.csv",
  # "HaibK562Yy1V0416101UniPk.csv",
  # "HaibK562Yy1V0416102UniPk.csv",
  # "HaibK562Zbtb33Pcr1xUniPk.csv",
  # "HaibK562Zbtb7asc34508V0416101UniPk.csv",
  # "HaibPanc1NrsfPcr2xUniPk.csv",
  # "HaibPanc1Pol24h8V0416101UniPk.csv",
  # "HaibPanc1Sin3ak20V0416101UniPk.csv",
  # "HaibPfsk1Foxp2Pcr2xUniPk.csv",
  # "HaibPfsk1NrsfPcr2xUniPk.csv",
  # "HaibPfsk1Sin3ak20V0416101UniPk.csv",
  # "HaibPfsk1Taf1V0416101UniPk.csv",
  # "HaibSknmcFoxp2Pcr2xUniPk.csv",
  # "HaibSknmcPol24h8V0416101UniPk.csv",
  # "HaibSknshNrsfPcr2xUniPk.csv",
  # "HaibSknshNrsfV0416101UniPk.csv",
  # "HaibSknshPol24h8V0416101UniPk.csv",
  # "HaibSknshSin3ak20V0416101UniPk.csv",
  # "HaibSknshTaf1V0416101UniPk.csv",
  # "HaibSknshraCtcfV0416102UniPk.csv",
  # "HaibSknshraP300V0416102UniPk.csv",
  # "HaibSknshraRad21V0416102UniPk.csv",
  # "HaibSknshraUsf1sc8983V0416102UniPk.csv",
  # "HaibSknshraYy1sc281V0416102UniPk.csv",
  # "HaibT47dCtcfsc5916V0416102Dm002p1hUniPk.csv",
  # "HaibT47dEraaV0416102Bpa1hUniPk.csv",
  # "HaibT47dEralphaaPcr2xGen1hUniPk.csv",
  # "HaibT47dEralphaaV0416102Est10nm1hUniPk.csv",
  # "HaibT47dFoxa1sc6553V0416102Dm002p1hUniPk.csv",
  # "HaibT47dGata3sc268V0416102Dm002p1hUniPk.csv",
  # "HaibT47dP300V0416102Dm002p1hUniPk.csv",
  # "HaibU87NrsfPcr2xUniPk.csv",
  # "HaibU87Pol24h8V0416101UniPk.csv",
  # "SydhA549Bhlhe40IggrabUniPk.csv",
  # "SydhA549CebpbIggrabUniPk.csv",
  # "SydhA549MaxIggrabUniPk.csv",
  # "SydhA549Pol2s2IggrabUniPk.csv",
  # "SydhA549Rad21IggrabUniPk.csv",
  # "SydhGm08714Znf274UcdUniPk.csv",
  # "SydhGm10847NfkbTnfaIggrabUniPk.csv",
  # "SydhGm10847Pol2IggmusUniPk.csv",
  # "SydhGm12878Bhlhe40cIggmusUniPk.csv",
  # "SydhGm12878Brca1a300IggmusUniPk.csv",
  # "SydhGm12878CfosUniPk.csv",
  # "SydhGm12878Chd1a301218aIggmusUniPk.csv",
  # "SydhGm12878Chd2ab68301IggmusUniPk.csv",
  # "SydhGm12878Corestsc30189IggmusUniPk.csv",
  # "SydhGm12878Ctcfsc15914c20UniPk.csv",
  # "SydhGm12878E2f4IggmusUniPk.csv",
  # "SydhGm12878Ebf1sc137065UniPk.csv",
  # "SydhGm12878Elk112771IggmusUniPk.csv",
  # "SydhGm12878Ikzf1iknuclaUniPk.csv",
  # "SydhGm12878JundUniPk.csv",
  # "SydhGm12878MaxIggmusUniPk.csv",
  # "SydhGm12878Mazab85725IggmusUniPk.csv",
  # "SydhGm12878Mxi1IggmusUniPk.csv",
  # "SydhGm12878Nfe2sc22827UniPk.csv",
  # "SydhGm12878NfkbTnfaIggrabUniPk.csv",
  # "SydhGm12878NfyaIggmusUniPk.csv",
  # "SydhGm12878NfybIggmusUniPk.csv",
  # "SydhGm12878Nrf1IggmusUniPk.csv",
  # "SydhGm12878P300IggmusUniPk.csv",
  # "SydhGm12878P300bUniPk.csv",
  # "SydhGm12878Pol2IggmusUniPk.csv",
  # "SydhGm12878Pol2UniPk.csv",
  # "SydhGm12878Pol2s2IggmusUniPk.csv",
  # "SydhGm12878Pol3UniPk.csv",
  # "SydhGm12878Rad21IggrabUniPk.csv",
  # "SydhGm12878Rfx5200401194IggmusUniPk.csv",
  # "SydhGm12878Sin3anb6001263IggmusUniPk.csv",
  # "SydhGm12878Smc3ab9263IggmusUniPk.csv",
  # "SydhGm12878Stat1UniPk.csv",
  # "SydhGm12878Stat3IggmusUniPk.csv",
  # "SydhGm12878Tblr1ab24550IggmusUniPk.csv",
  # "SydhGm12878TbpIggmusUniPk.csv",
  # "SydhGm12878Tr4UniPk.csv",
  # "SydhGm12878Usf2IggmusUniPk.csv",
  # "SydhGm12878WhipIggmusUniPk.csv",
  # "SydhGm12878Yy1UniPk.csv",
  # "SydhGm12878Znf143166181apUniPk.csv",
  # "SydhGm12878Znf274UniPk.csv",
  # "SydhGm12878Zzz3UniPk.csv",
  # "SydhGm12891NfkbTnfaIggrabUniPk.csv",
  # "SydhGm12891Pol2IggmusUniPk.csv",
  # "SydhGm12892NfkbTnfaIggrabUniPk.csv",
  # "SydhGm12892Pol2IggmusUniPk.csv",
  # "SydhGm15510NfkbTnfaIggrabUniPk.csv",
  # "SydhGm15510Pol2IggmusUniPk.csv",
  # "SydhGm18505NfkbTnfaIggrabUniPk.csv",
  # "SydhGm18505Pol2IggmusUniPk.csv",
  # "SydhGm18526NfkbTnfaIggrabUniPk.csv",
  # "SydhGm18526Pol2IggmusUniPk.csv",
  # "SydhGm18951NfkbTnfaIggrabUniPk.csv",
  # "SydhGm18951Pol2IggmusUniPk.csv",
  # "SydhGm19099NfkbTnfaIggrabUniPk.csv",
  # "SydhGm19099Pol2IggmusUniPk.csv",
  # "SydhGm19193NfkbTnfaIggrabUniPk.csv",
  # "SydhGm19193Pol2IggmusUniPk.csv",
  # "SydhH1hescBach1sc14700IggrabUniPk.csv",
  # "SydhH1hescBrca1IggrabUniPk.csv",
  # "SydhH1hescCebpbIggrabUniPk.csv",
  # "SydhH1hescChd1a301218aIggrabUniPk.csv",
  # "SydhH1hescChd2IggrabUniPk.csv",
  # "SydhH1hescCjunIggrabUniPk.csv",
  # "SydhH1hescCmycIggrabUniPk.csv",
  # "SydhH1hescCtbp2UcdUniPk.csv",
  # "SydhH1hescGtf2f1IggrabUniPk.csv",
  # "SydhH1hescJundIggrabUniPk.csv",
  # "SydhH1hescMafkIggrabUniPk.csv",
  # "SydhH1hescMaxUcdUniPk.csv",
  # "SydhH1hescMxi1IggrabUniPk.csv",
  # "SydhH1hescNrf1IggrabUniPk.csv",
  # "SydhH1hescRad21IggrabUniPk.csv",
  # "SydhH1hescRfx5200401194IggrabUniPk.csv",
  # "SydhH1hescSin3anb6001263IggrabUniPk.csv",
  # "SydhH1hescSuz12UcdUniPk.csv",
  # "SydhH1hescTbpIggrabUniPk.csv",
  # "SydhH1hescUsf2IggrabUniPk.csv",
  # "SydhH1hescZnf143IggrabUniPk.csv",
  # "SydhHct116Pol2UcdUniPk.csv",
  # "SydhHct116Tcf7l2UcdUniPk.csv",
  # "SydhHek293Elk4UcdUniPk.csv",
  # "SydhHek293Kap1UcdUniPk.csv",
  # "SydhHek293Pol2UniPk.csv",
  # "SydhHek293Tcf7l2UcdUniPk.csv",
  # "SydhHek293tZnf263UcdUniPk.csv",
  # "SydhHelas3Ap2alphaUniPk.csv",
  # "SydhHelas3Ap2gammaUniPk.csv",
  # "SydhHelas3Baf155IggmusUniPk.csv",
  # "SydhHelas3Baf170IggmusUniPk.csv",
  # "SydhHelas3Bdp1UniPk.csv",
  # "SydhHelas3Brca1a300IggrabUniPk.csv",
  # "SydhHelas3Brf1UniPk.csv",
  # "SydhHelas3Brf2UniPk.csv",
  # "SydhHelas3Brg1IggmusUniPk.csv",
  # "SydhHelas3CebpbIggrabUniPk.csv",
  # "SydhHelas3CfosUniPk.csv",
  # "SydhHelas3Chd2IggrabUniPk.csv",
  # "SydhHelas3CjunIggrabUniPk.csv",
  # "SydhHelas3CmycUniPk.csv",
  # "SydhHelas3Corestsc30189IggrabUniPk.csv",
  # "SydhHelas3E2f1UniPk.csv",
  # "SydhHelas3E2f4UniPk.csv",
  # "SydhHelas3E2f6UniPk.csv",
  # "SydhHelas3Elk112771IggrabUniPk.csv",
  # "SydhHelas3Elk4UcdUniPk.csv",
  # "SydhHelas3Gtf2f1ab28179IggrabUniPk.csv",
  # "SydhHelas3Hae2f1UniPk.csv",
  # "SydhHelas3Ini1IggmusUniPk.csv",
  # "SydhHelas3Irf3IggrabUniPk.csv",
  # "SydhHelas3JundIggrabUniPk.csv",
  # "SydhHelas3MafkIggrabUniPk.csv",
  # "SydhHelas3MaxIggrabUniPk.csv",
  # "SydhHelas3Mazab85725IggrabUniPk.csv",
  # "SydhHelas3Mxi1af4185IggrabUniPk.csv",
  # "SydhHelas3NfyaIggrabUniPk.csv",
  # "SydhHelas3NfybIggrabUniPk.csv",
  # "SydhHelas3Nrf1IggmusUniPk.csv",
  # "SydhHelas3P300sc584sc584IggrabUniPk.csv",
  # "SydhHelas3Pol2UniPk.csv",
  # "SydhHelas3Pol2s2IggrabUniPk.csv",
  # "SydhHelas3Prdm19115IggrabUniPk.csv",
  # "SydhHelas3Rad21IggrabUniPk.csv",
  # "SydhHelas3Rfx5200401194IggrabUniPk.csv",
  # "SydhHelas3Rpc155UniPk.csv",
  # "SydhHelas3Smc3ab9263IggrabUniPk.csv",
  # "SydhHelas3Spt20UniPk.csv",
  # "SydhHelas3Stat1Ifng30UniPk.csv",
  # "SydhHelas3Stat3IggrabUniPk.csv",
  # "SydhHelas3TbpIggrabUniPk.csv",
  # "SydhHelas3Tcf7l2UcdUniPk.csv",
  # "SydhHelas3Tcf7l2c9b92565UcdUniPk.csv",
  # "SydhHelas3Tf3c110UniPk.csv",
  # "SydhHelas3Tr4UniPk.csv",
  # "SydhHelas3Usf2IggmusUniPk.csv",
  # "SydhHelas3Zkscan1hpa006672IggrabUniPk.csv",
  # "SydhHelas3Znf143IggrabUniPk.csv",
  # "SydhHelas3Znf274UcdUniPk.csv",
  # "SydhHelas3Zzz3UniPk.csv",
  # "SydhHepg2Arid3anb100279IggrabUniPk.csv",
  # "SydhHepg2Bhlhe40cIggrabUniPk.csv",
  # "SydhHepg2Brca1a300IggrabUniPk.csv",
  # "SydhHepg2CebpbForsklnUniPk.csv",
  # "SydhHepg2CebpbIggrabUniPk.csv",
  # "SydhHepg2Chd2ab68301IggrabUniPk.csv",
  # "SydhHepg2CjunIggrabUniPk.csv",
  # "SydhHepg2Corestsc30189IggrabUniPk.csv",
  # "SydhHepg2ErraForsklnUniPk.csv",
  # "SydhHepg2Grp20ForsklnUniPk.csv",
  # "SydhHepg2Hnf4aForsklnUniPk.csv",
  # "SydhHepg2Hsf1ForsklnUniPk.csv",
  # "SydhHepg2Irf3IggrabUniPk.csv",
  # "SydhHepg2JundIggrabUniPk.csv",
  # "SydhHepg2Maffm8194IggrabUniPk.csv",
  # "SydhHepg2Mafkab50322IggrabUniPk.csv",
  # "SydhHepg2Mafksc477IggrabUniPk.csv",
  # "SydhHepg2MaxIggrabUniPk.csv",
  # "SydhHepg2Mazab85725IggrabUniPk.csv",
  # "SydhHepg2Mxi1UniPk.csv",
  # "SydhHepg2Nrf1IggrabUniPk.csv",
  # "SydhHepg2P300sc582IggrabUniPk.csv",
  # "SydhHepg2Pgc1aForsklnUniPk.csv",
  # "SydhHepg2Pol2ForsklnUniPk.csv",
  # "SydhHepg2Pol2IggrabUniPk.csv",
  # "SydhHepg2Pol2s2IggrabUniPk.csv",
  # "SydhHepg2Rad21IggrabUniPk.csv",
  # "SydhHepg2Rfx5200401194IggrabUniPk.csv",
  # "SydhHepg2Smc3ab9263IggrabUniPk.csv",
  # "SydhHepg2Srebp1InslnUniPk.csv",
  # "SydhHepg2TbpIggrabUniPk.csv",
  # "SydhHepg2Tcf7l2UcdUniPk.csv",
  # "SydhHepg2Tr4UcdUniPk.csv",
  # "SydhHepg2Usf2IggrabUniPk.csv",
  # "SydhHepg2Znf274UcdUniPk.csv",
  # "SydhHuvecCfosUcdUniPk.csv",
  # "SydhHuvecCjunUniPk.csv",
  # "SydhHuvecGata2UcdUniPk.csv",
  # "SydhHuvecMaxUniPk.csv",
  # "SydhHuvecPol2UniPk.csv",
  # "SydhImr90CebpbIggrabUniPk.csv",
  # "SydhImr90CtcfbIggrabUniPk.csv",
  # "SydhImr90MafkIggrabUniPk.csv",
  # "SydhImr90Pol2IggrabUniPk.csv",
  # "SydhImr90Rad21IggrabUniPk.csv",
  # "SydhK562Arid3asc8821IggrabUniPk.csv",
  # "SydhK562Atf106325UniPk.csv",
  # "SydhK562Atf3UniPk.csv",
  # "SydhK562Bach1sc14700IggrabUniPk.csv",
  # "SydhK562Bdp1UniPk.csv",
  # "SydhK562Bhlhe40nb100IggrabUniPk.csv",
  # "SydhK562Brf1UniPk.csv",
  # "SydhK562Brf2UniPk.csv",
  # "SydhK562Brg1IggmusUniPk.csv",
  # "SydhK562Ccnt2UniPk.csv",
  # "SydhK562CebpbIggrabUniPk.csv",
  # "SydhK562CfosUniPk.csv",
  # "SydhK562Chd2ab68301IggrabUniPk.csv",
  # "SydhK562CjunIfna30UniPk.csv",
  # "SydhK562CjunIfna6hUniPk.csv",
  # "SydhK562CjunIfng30UniPk.csv",
  # "SydhK562CjunIfng6hUniPk.csv",
  # "SydhK562CjunUniPk.csv",
  # "SydhK562CmycIfna30UniPk.csv",
  # "SydhK562CmycIfna6hUniPk.csv",
  # "SydhK562CmycIfng30UniPk.csv",
  # "SydhK562CmycIfng6hUniPk.csv",
  # "SydhK562CmycIggrabUniPk.csv",
  # "SydhK562CmycUniPk.csv",
  # "SydhK562Corestab24166IggrabUniPk.csv",
  # "SydhK562Corestsc30189IggrabUniPk.csv",
  # "SydhK562CtcfbIggrabUniPk.csv",
  # "SydhK562E2f4UcdUniPk.csv",
  # "SydhK562E2f6UcdUniPk.csv",
  # "SydhK562Elk112771IggrabUniPk.csv",
  # "SydhK562Gata1UcdUniPk.csv",
  # "SydhK562Gata2UcdUniPk.csv",
  # "SydhK562Gtf2bUniPk.csv",
  # "SydhK562Gtf2f1ab28179IggrabUniPk.csv",
  # "SydhK562Hmgn3UniPk.csv",
  # "SydhK562Ini1IggmusUniPk.csv",
  # "SydhK562Irf1Ifna30UniPk.csv",
  # "SydhK562Irf1Ifna6hUniPk.csv",
  # "SydhK562Irf1Ifng30UniPk.csv",
  # "SydhK562Irf1Ifng6hUniPk.csv",
  # "SydhK562JundIggrabUniPk.csv",
  # "SydhK562Kap1UcdUniPk.csv",
  # "SydhK562MaffIggrabUniPk.csv",
  # "SydhK562Mafkab50322IggrabUniPk.csv",
  # "SydhK562MaxIggrabUniPk.csv",
  # "SydhK562Mazab85725IggrabUniPk.csv",
  # "SydhK562Mxi1af4185IggrabUniPk.csv",
  # "SydhK562NelfeUniPk.csv",
  # "SydhK562Nfe2UniPk.csv",
  # "SydhK562NfyaUniPk.csv",
  # "SydhK562NfybUniPk.csv",
  # "SydhK562Nrf1IggrabUniPk.csv",
  # "SydhK562P300IggrabUniPk.csv",
  # "SydhK562Pol2Ifna30UniPk.csv",
  # "SydhK562Pol2Ifna6hUniPk.csv",
  # "SydhK562Pol2Ifng30UniPk.csv",
  # "SydhK562Pol2Ifng6hUniPk.csv",
  # "SydhK562Pol2IggmusUniPk.csv",
  # "SydhK562Pol2UniPk.csv",
  # "SydhK562Pol2s2IggrabUniPk.csv",
  # "SydhK562Pol2s2UniPk.csv",
  # "SydhK562Pol3UniPk.csv",
  # "SydhK562Rad21UniPk.csv",
  # "SydhK562Rfx5IggrabUniPk.csv",
  # "SydhK562Rpc155UniPk.csv",
  # "SydhK562Setdb1MnasedUcdUniPk.csv",
  # "SydhK562Setdb1UcdUniPk.csv",
  # "SydhK562Sirt6UniPk.csv",
  # "SydhK562Smc3ab9263IggrabUniPk.csv",
  # "SydhK562Stat1Ifna30UniPk.csv",
  # "SydhK562Stat1Ifna6hUniPk.csv",
  # "SydhK562Stat1Ifng30UniPk.csv",
  # "SydhK562Stat1Ifng6hUniPk.csv",
  # "SydhK562Stat2Ifna30UniPk.csv",
  # "SydhK562Stat2Ifna6hUniPk.csv",
  # "SydhK562Tal1sc12984IggmusUniPk.csv",
  # "SydhK562Tblr1ab24550IggrabUniPk.csv",
  # "SydhK562Tblr1nb600270IggrabUniPk.csv",
  # "SydhK562TbpIggmusUniPk.csv",
  # "SydhK562Tf3c110UniPk.csv",
  # "SydhK562Tr4UcdUniPk.csv",
  # "SydhK562Ubfsc13125IggmusUniPk.csv",
  # "SydhK562Ubtfsab1404509IggmusUniPk.csv",
  # "SydhK562Usf2IggrabUniPk.csv",
  # "SydhK562Yy1UcdUniPk.csv",
  # "SydhK562Znf143IggrabUniPk.csv",
  # "SydhK562Znf263UcdUniPk.csv",
  # "SydhK562Znf274UcdUniPk.csv",
  # "SydhK562Znf274m01UcdUniPk.csv",
  # "SydhMcf10aesCfosEtoh01HvdUniPk.csv",
  # "SydhMcf10aesCfosTam112hHvdUniPk.csv",
  # "SydhMcf10aesCfosTam14hHvdUniPk.csv",
  # "SydhMcf10aesCfosTamHvdUniPk.csv",
  # "SydhMcf10aesCmycEtoh01HvdUniPk.csv",
  # "SydhMcf10aesCmycTam14hHvdUniPk.csv",
  # "SydhMcf10aesE2f4TamHvdUniPk.csv",
  # "SydhMcf10aesPol2Etoh01UniPk.csv",
  # "SydhMcf10aesPol2TamUniPk.csv",
  # "SydhMcf10aesStat3Etoh01UniPk.csv",
  # "SydhMcf10aesStat3Etoh01bUniPk.csv",
  # "SydhMcf10aesStat3Etoh01cUniPk.csv",
  # "SydhMcf10aesStat3Tam112hHvdUniPk.csv",
  # "SydhMcf10aesStat3TamUniPk.csv",
  # "SydhMcf7Gata3UcdUniPk.csv",
  # "SydhMcf7Gata3sc269UcdUniPk.csv",
  # "SydhMcf7Hae2f1UcdUniPk.csv",
  # "SydhMcf7Tcf7l2UcdUniPk.csv",
  # "SydhMcf7Znf217UcdUniPk.csv",
  # "SydhNb4CmycUniPk.csv",
  # "SydhNb4MaxUniPk.csv",
  # "SydhNb4Pol2UniPk.csv",
  # "SydhNt2d1Suz12UcdUniPk.csv",
  # "SydhNt2d1Yy1UcdUniPk.csv",
  # "SydhNt2d1Znf274UcdUniPk.csv",
  # "SydhPanc1Tcf7l2UcdUniPk.csv",
  # "SydhPbdeGata1UcdUniPk.csv",
  # "SydhPbdePol2UcdUniPk.csv",
  # "SydhPbdefetalGata1UcdUniPk.csv",
  # "SydhRajiPol2UcdUniPk.csv",
  # "SydhShsy5yGata2UcdUniPk.csv",
  # "SydhShsy5yGata3sc269sc269UcdUniPk.csv",
  # "SydhU2osKap1UcdUniPk.csv",
  # "SydhU2osSetdb1UcdUniPk.csv",
  # "UchicagoK562EfosUniPk.csv",
  # "UchicagoK562Egata2UniPk.csv",
  # "UchicagoK562Ehdac8UniPk.csv",
  # "UchicagoK562EjunbUniPk.csv",
  # "UchicagoK562EjundUniPk.csv",
  # "UtaA549CtcfUniPk.csv",
  # "UtaA549Pol2UniPk.csv",
  # "UtaFibroblCtcfUniPk.csv",
  # "UtaGlioblaCtcfUniPk.csv",
  # "UtaGlioblaPol2UniPk.csv",
  # "UtaGm12878CmycUniPk.csv",
  # "UtaGm12878CtcfUniPk.csv",
  # "UtaGm12878Pol2UniPk.csv",
  # "UtaGm12891CtcfUniPk.csv",
  # "UtaGm12892CtcfUniPk.csv",
  # "UtaGm19238CtcfUniPk.csv",
  # "UtaGm19239CtcfUniPk.csv",
  # "UtaGm19240CtcfUniPk.csv",
  # "UtaH1hescCmycUniPk.csv",
  # "UtaH1hescCtcfUniPk.csv",
  # "UtaH1hescPol2UniPk.csv",
  # "UtaHelas3CmycUniPk.csv",
  # "UtaHelas3CtcfUniPk.csv",
  # "UtaHelas3Pol2UniPk.csv",
  # "UtaHepg2CmycUniPk.csv",
  # "UtaHepg2CtcfUniPk.csv",
  # "UtaHepg2Pol2UniPk.csv",
  # "UtaHuvecCmycUniPk.csv",
  # "UtaHuvecCtcfUniPk.csv",
  # "UtaHuvecPol2UniPk.csv",
  # "UtaK562CmycUniPk.csv",
  # "UtaK562CtcfUniPk.csv",
  # "UtaK562Pol2UniPk.csv",
  # "UtaMcf7CmycEstroUniPk.csv",
  # "UtaMcf7CmycSerumstimUniPk.csv",
  # "UtaMcf7CmycSerumstvdUniPk.csv",
  # "UtaMcf7CmycVehUniPk.csv",
  # "UtaMcf7CtcfEstroUniPk.csv",
  # "UtaMcf7CtcfSerumstimUniPk.csv",
  # "UtaMcf7CtcfSerumstvdUniPk.csv",
  # "UtaMcf7CtcfUniPk.csv",
  # "UtaMcf7CtcfVehUniPk.csv",
  # "UtaMcf7Pol2SerumstimUniPk.csv",
  # "UtaMcf7Pol2SerumstvdUniPk.csv",
  # "UtaMcf7Pol2UniPk.csv",
  # "UtaNhekCtcfUniPk.csv",
  # "UtaProgfibCtcfUniPk.csv",
  # "UtaProgfibPol2UniPk.csv",
  # "UwA549CtcfUniPk.csv",
  # "UwAg04449CtcfUniPk.csv",
  # "UwAg04450CtcfUniPk.csv",
  # "UwAg09309CtcfUniPk.csv",
  # "UwAg09319CtcfUniPk.csv",
  # "UwAg10803CtcfUniPk.csv",
  # "UwAoafCtcfUniPk.csv",
  # "UwBe2cCtcfUniPk.csv",
  # "UwBjCtcfUniPk.csv",
  # "UwCaco2CtcfUniPk.csv",
  # "UwGm06990CtcfUniPk.csv",
  # "UwGm12801CtcfUniPk.csv",
  # "UwGm12864CtcfUniPk.csv",
  # "UwGm12865CtcfUniPk.csv",
  # "UwGm12872CtcfUniPk.csv",
  # "UwGm12873CtcfUniPk.csv",
  # "UwGm12874CtcfUniPk.csv",
  # "UwGm12875CtcfUniPk.csv",
  # "UwGm12878CtcfUniPk.csv",
  # "UwHacCtcfUniPk.csv",
  # "UwHaspCtcfUniPk.csv",
  # "UwHbmecCtcfUniPk.csv",
  # "UwHcfaaCtcfUniPk.csv",
  # "UwHcmCtcfUniPk.csv",
  # "UwHcpeCtcfUniPk.csv",
  # "UwHct116CtcfUniPk.csv",
  # "UwHeeCtcfUniPk.csv",
  # "UwHek293CtcfUniPk.csv",
  # "UwHelas3CtcfUniPk.csv",
  # "UwHepg2CtcfUniPk.csv",
  # "UwHffCtcfUniPk.csv",
  # "UwHffmycCtcfUniPk.csv",
  # "UwHl60CtcfUniPk.csv",
  # "UwHmecCtcfUniPk.csv",
  # "UwHmfCtcfUniPk.csv",
  # "UwHpafCtcfUniPk.csv",
  # "UwHpfCtcfUniPk.csv",
  # "UwHreCtcfUniPk.csv",
  # "UwHrpeCtcfUniPk.csv",
  # "UwHuvecCtcfUniPk.csv",
  # "UwHvmfCtcfUniPk.csv",
  # "UwK562CtcfUniPk.csv",
  # "UwMcf7CtcfUniPk.csv",
  # "UwNb4CtcfUniPk.csv",
  # "UwNhdfneoCtcfUniPk.csv",
  # "UwNhekCtcfUniPk.csv",
  # "UwNhlfCtcfUniPk.csv",
  # "UwRptecCtcfUniPk.csv",
  # "UwSaecCtcfUniPk.csv",
  # "UwSknshraCtcfUniPk.csv",
  # "UwWerirb1CtcfUniPk.csv",
  # "UwWi38CtcfUniPk.csv",
# ]:

  dsname = "/content/drive/MyDrive/DNABERT/Datasets/tfbs/" + fname
  tfbs_dataset = pd.read_csv(dsname, sep=',')

# change standard error and output to saved output file

  stdpath = "/content/drive/MyDrive/DNABERT/output/" + fname.split(".")[0] + "/"

  try:
    os.makedirs(stdpath)
  except FileExistsError:
    pass

  stdpath_out = stdpath + "stdout.txt"
  stdpath_err = stdpath + "stderr.txt"

  sys.stdout = open(stdpath_out, 'w')
  sys.stderr = open(stdpath_err, 'w')

# print dataset shape and initial values

  print("tfbs dataset:", tfbs_dataset.shape, file=sys.stdout)
  print("tfbs dataset initial values:", tfbs_dataset.head(), file=sys.stdout)

# reformat input

  column_names = ["labels", "seq"]
  tfbs_dataset_res = pd.DataFrame(columns=column_names)

  j = 1
  for i in range(tfbs_dataset.count()['names']):
    name = tfbs_dataset['names'][i]
    pos_seq = tfbs_dataset['seq'][i]
    neg_seq = tfbs_dataset['neg_seq'][i]
    pos_label = 1
    neg_label = 0
    tfbs_dataset_res.loc[j] = [pos_label, pos_seq]
    tfbs_dataset_res.loc[j+1] = [neg_label, neg_seq]
    j+=2

  X = tfbs_dataset_res['seq']
  y = tfbs_dataset_res['labels']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# print size of training and test sets

  print("X_train shape:", X_train.shape, file=sys.stdout)
  print("y_train shape:", y_train.shape, file=sys.stdout)
  print("X_test shape:", X_test.shape, file=sys.stdout)
  print("y_test shape:", y_test.shape, file=sys.stdout)

# tokenization

  def kmers_stride1(s, k=6):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

# load pre-trained model

  model_cls = AutoModelForSequenceClassification.from_pretrained("armheb/DNA_bert_6", num_labels=2)
  params = list(model_cls.named_parameters())
  tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

# reformat data to Hugging Face Dataset format from pandas

  ds_Xy_train = pd.concat([y_train, X_train], axis=1)
  ds_Xy_test = pd.concat([y_test, X_test], axis=1)

  Dataset_Xy_train = Dataset.from_pandas(ds_Xy_train)
  Dataset_Xy_test = Dataset.from_pandas(ds_Xy_test)
  Dataset_Xy_train, Dataset_Xy_test

  def tok_func(x): return tokenizer(" ".join(kmers_stride1(x["seq"])))

  Dataset_Xy_train_tok = Dataset_Xy_train.map(tok_func, batched=False)
  new_column = ["train"] * len(Dataset_Xy_train_tok)
  Dataset_Xy_train_tok = Dataset_Xy_train_tok.add_column("dset", new_column)

  Dataset_Xy_test_tok = Dataset_Xy_test.map(tok_func, batched=False)
  new_column = ["test"] * len(Dataset_Xy_test_tok)
  Dataset_Xy_test_tok = Dataset_Xy_test_tok.add_column("dset", new_column)

  dds = DatasetDict({
    'train': Dataset_Xy_train_tok,
    'test': Dataset_Xy_test_tok
  })

# switch to GPU

  if torch.cuda.device_count() > 0:
    model_cls.to('cuda')

# train model

  bs = 32
  epochs = 4
  lr = 8e-5

  args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

  def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

  trainer.train();

# save model

  fpath = "/content/drive/MyDrive/DNABERT/Output_Models/" + fname.split(".")[0] + "/"
  print(fpath, file=sys.stdout)
  trainer.save_model(fpath)

# test model

  eval_preds = trainer.predict(dds['test'])

  print(eval_preds, file=sys.stdout)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_6 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/77988 [00:00<?, ? examples/s]

Map:   0%|          | 0/25996 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.296700,0.253589,0.902293,0.898950
2,0.233500,0.270653,0.905716,0.906788
3,0.173200,0.287622,0.901216,0.904265


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.296700,0.253589,0.902293,0.898950
2,0.233500,0.270653,0.905716,0.906788
3,0.173200,0.287622,0.901216,0.904265
4,0.117900,0.407823,0.889945,0.895473
